In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/Vision-CAIR/MiniGPT-4.git

fatal: destination path 'MiniGPT-4' already exists and is not an empty directory.


In [ ]:
ls

drive/  img2.png  MiniGPT-4/  prerained_minigpt4_7b.pth  recipe1M_images/  sample_data/


In [ ]:
%cd MiniGPT-4/

/content/MiniGPT-4


In [ ]:
!pip install -qr requirements.txt
!pip install -q salesforce-lavis
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q h5py
!pip install -q typing-extensions
!pip install -q wheel
!pip install -q git+https://github.com/huggingface/transformers.git -U
!pip install "accelerate>=0.20.3" --q
!pip install visual_genome --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 48.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 1

In [ ]:
!wget https://huggingface.co/wangrongsheng/MiniGPT4-7B/resolve/main/prerained_minigpt4_7b.pth

--2023-10-30 21:21:08--  https://huggingface.co/wangrongsheng/MiniGPT4-7B/resolve/main/prerained_minigpt4_7b.pth
Resolving huggingface.co (huggingface.co)... 65.8.178.93, 65.8.178.27, 65.8.178.12, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.93|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/25/fa/25fa880e116eda3d82c5c87b32eb0a7fa8f76b139d70ce756851215ce7a76179/017a9ed588a11ed383711003cf50cf675191420a04689f682fb56fa9bbb8dcbb?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27prerained_minigpt4_7b.pth%3B+filename%3D%22prerained_minigpt4_7b.pth%22%3B&Expires=1698960069&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5ODk2MDA2OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8yNS9mYS8yNWZhODgwZTExNmVkYTNkODJjNWM4N2IzMmViMGE3ZmE4Zjc2YjEzOWQ3MGNlNzU2ODUxMjE1Y2U3YTc2MTc5LzAxN2E5ZWQ1ODhhMTFlZDM4MzcxMTAwM2NmNTBjZjY3NTE5MTQyMGEwNDY4OWY2OD

In [ ]:
import argparse
import os
import random

import numpy as np
import torch
import torch.backends.cudnn as cudnn
from transformers import StoppingCriteriaList

from minigpt4.common.config import Config
from minigpt4.common.dist_utils import get_rank
from minigpt4.common.registry import registry
from minigpt4.conversation.conversation import Chat, CONV_VISION_Vicuna0, CONV_VISION_LLama2, StoppingCriteriaSub

from minigpt4.datasets.builders import *
from minigpt4.models import *
from minigpt4.processors import *
from minigpt4.runners import *
from minigpt4.tasks import *
import argparse
import time
from threading import Thread
from PIL import Image

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

import dataclasses
from enum import auto, Enum
from typing import List, Tuple, Any

from minigpt4.common.registry import registry
import pandas as pd
from tqdm import tqdm

In [ ]:
class Args:
    def __init__(self, cfg_path, gpu_id=0, options=None):
        self.cfg_path = cfg_path
        self.gpu_id = gpu_id
        self.options = options
args = Args(cfg_path="eval_configs/minigpt4_eval.yaml", gpu_id=0)

In [ ]:
def setup_seeds(config):
    seed = config.run_cfg.seed + get_rank()

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    cudnn.benchmark = False
    cudnn.deterministic = True

In [ ]:
conv_dict = {'pretrain_vicuna0': CONV_VISION_Vicuna0,
             'pretrain_llama2': CONV_VISION_LLama2}

cfg = Config(args)

model_config = cfg.model_cfg
model_config.device_8bit = args.gpu_id
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config).to('cuda:{}'.format(args.gpu_id))

CONV_VISION = conv_dict[model_config.model_type]

vis_processor_cfg = cfg.datasets_cfg.cc_sbu_align.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)

stop_words_ids = [[835], [2277, 29937]]
stop_words_ids = [torch.tensor(ids).to(device='cuda:{}'.format(args.gpu_id)) for ids in stop_words_ids]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [ ]:
def encode_img(img_list):
        image = img_list[0]
        img_list.pop(0)
        if isinstance(image, str):  # is a image path
            raw_image = Image.open(image).convert('RGB')
            image = vis_processor(raw_image).unsqueeze(0).to(device)
        elif isinstance(image, Image.Image):
            raw_image = image
            image = vis_processor(raw_image).unsqueeze(0).to(device)
        elif isinstance(image, torch.Tensor):
            if len(image.shape) == 3:
                image = image.unsqueeze(0)
            image = image.to(device)

        image_emb, _ = model.encode_img(image)
        img_list.append(image_emb)
        return img_list

In [ ]:
def answer_prepare(prompt, img_list, max_new_tokens=300, num_beams=1, min_length=1, top_p=0.9,
                       repetition_penalty=1.05, length_penalty=1, temperature=1.0, max_length=2000):
    embs = get_context_emb(prompt, img_list)
    current_max_len = embs.shape[1] + max_new_tokens
    if current_max_len - max_length > 0:
        print('Warning: The number of tokens in current conversation exceeds the max length. '
                'The model will not see the contexts outside the range.')
    begin_idx = max(0, current_max_len - max_length)
    embs = embs[:, begin_idx:]

    generation_kwargs = dict(
        inputs_embeds=embs,
        max_new_tokens=max_new_tokens,
        stopping_criteria=stopping_criteria,
        num_beams=num_beams,
        do_sample=True,
        min_length=min_length,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        temperature=float(temperature),
    )
    return generation_kwargs

In [ ]:
def get_context_emb(prompt, img_list):
    prompt_segs = prompt.split('<ImageHere>')
    assert len(prompt_segs) == len(img_list) + 1, "Unmatched numbers of image placeholders and images."
    seg_tokens = [
        model.llama_tokenizer(
            seg, return_tensors="pt", add_special_tokens=i == 0).to(device).input_ids
        # only add bos to the first seg
        for i, seg in enumerate(prompt_segs)
    ]
    seg_embs = [model.embed_tokens(seg_t) for seg_t in seg_tokens]
    mixed_embs = [emb for pair in zip(seg_embs[:-1], img_list) for emb in pair] + [seg_embs[-1]]
    mixed_embs = torch.cat(mixed_embs, dim=1)
    return mixed_embs

In [ ]:
def answer(prompt, img_list, **kargs):
    generation_dict = answer_prepare(prompt, img_list, **kargs)

    output_token = model.llama_model.generate(**generation_dict)[0]
    output_text = model.llama_tokenizer.decode(output_token, skip_special_tokens=True)

    output_text = output_text.split('###')[0]  # remove the stop sign '###'
    output_text = output_text.split('Assistant:')[-1].strip()

    return output_text, output_token.cpu().numpy()

In [ ]:
text = answer(prompt = prompt,
            img_list=img_list,
            num_beams= 1,
            temperature= 1.0,
            max_new_tokens=300,
            max_length=2000)[0]

In [ ]:
print(text)

There are two pork chops on a bun, with pineapple slices on top of them and served with barbecue sauce. The pineapple slices are cut into small wedges.
Recipe: BBQ Pork Chop Sandwich with Honey Mustard Sauce
Ingredients:
* 4 pork chops
* 4 buns
* 1/2 cup honey mustard sauce
* 4 pineapple wedges
* 2 tablespoons barbecue sauce
* Salt and pepper
Instructions:
1. Preheat the grill to medium heat.
2. Season the pork chops with salt and pepper.
3. Place the pork chops on the grill and cook for about 5 minutes per side, or until they are cooked through.
4. While the pork chops are cooking, prepare the honey mustard sauce by combining the honey, mustard, and cumin in a small bowl.
5. Once the pork chops are done, remove them from the grill and place them on the buns.
6. Top each sandwich with a tablespoon of barbecue sauce and a pineapple wedge.
7. Serve immediately.
8. Enjoy your delicious BBQ Pork Ch


In [ ]:
dataframe = pd.read_csv("/content/drive/MyDrive/11-777-MMML/data.csv")
dataframe.head()

,image_path,food_title,ingredients,instructions,id,split
0,val/f/f/f/3/fff3e1751a.jpg,Irresistible Peanut Butter Cookies,34 cup creamy peanut butter /t 12 cup Crisco s...,"Combine peanut butter, Crisco, brown sugar, mi...",0006354bc3,train
1,val/c/0/9/8/c09820985d.jpg,Sachertorte Cookies,"1 cup margarine or 1 cup butter, softened /t 1...",Heat oven to 325 degrees . /t In large bowl co...,0008b8f5bc,train
2,val/8/b/d/2/8bd27b65f3.jpg,Sachertorte Cookies,"1 cup margarine or 1 cup butter, softened /t 1...",Heat oven to 325 degrees . /t In large bowl co...,0008b8f5bc,train
3,val/8/2/c/e/82ce36880e.jpg,Slow Cooker French Onion Soup,"4 Tablespoons Butter /t 6 whole Yellow Onions,...","Melt the butter in a large skillet, then add t...",000d39d105,train
4,val/a/1/8/0/a180ab276a.jpg,West African Banana Fritters,"2 eggs, lightly beaten /t 12 cup cold milk /t ...",Heat 3 inches of oil in a heavy saucepan or de...,00195713fa,train


In [ ]:
val_images = dataframe[dataframe.split == 'val']['image_path'].apply(lambda x:"/content/recipe1M_images/" + x)
condition = "Generate a recipe based on this image, listing ingredients and instructions."
condition_h = "Generate a healthier recipe based on this image, listing ingredients and instructions."
recipe_o = []
recipe_h = []

for i in tqdm(range(1000)):
    img_list = [val_images.iloc[i]]
    img_list = encode_img(img_list)
    prompt = "Human: <Img><ImageHere></Img>" + condition
    prompt_h = "Human: <Img><ImageHere></Img>" + condition_h
    recipe_o.append(answer(prompt = prompt,
                    img_list=img_list,
                    num_beams= 1,
                    temperature= 1.0,
                    max_new_tokens=300,
                    max_length=2000)[0])
    recipe_h.append(answer(prompt = prompt_h,
                    img_list=img_list,
                    num_beams= 1,
                    temperature= 1.0,
                    max_new_tokens=300,
                    max_length=2000)[0])